In [ ]:
pip install glob2

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.image as image
import glob2
import os

In [ ]:
images_fp="./images"

In [ ]:
image_names=[os.path.basename(file) for file in glob.glob(os.path.join(images_fp,'*.jpg'))]

In [ ]:
image_names

In [ ]:
labels=[' '.join(name.split('_')[:-1:]) for name in image_names]

In [ ]:
labels

In [ ]:
len(set(labels))

In [ ]:
def label_encoder(label):
    if label=='Abyssinian': return 0
    elif label=='Bengal': return 1
    elif label=='Birman': return 2
    elif label=='Bombay': return 3
    elif label=='British Shorthair': return 4
    elif label=='Egyptian Mau': return 5
    elif label=='american bull dog': return 6
    elif label=='americam pit bull terrier': return 7
    elif label=='bassat hound': return 8
    elif label=='beagle': return 9
    elif label=='boxer': return 10
    elif label=='chihuahua': return 11
    elif label=='english cocker spaniel': return 12
    elif label=='english setter': return 13
    elif label=='german shorthair': return 14

In [ ]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import load_img,img_to_array

In [ ]:
features = []
labels = []
IMAGE_SIZE = (224, 224)

for name in image_names:
    label = ''.join(name.split('_')[:-1:])
    label_encoded = label_encoder(label)  
    if label_encoded != None:
        img = load_img(os.path.join(images_fp, name)) 
        img = tf.image.resize_with_pad(img_to_array(img, dtype='uint8'), *IMAGE_SIZE).numpy().astype('uint8')
        img = np.array(img)
        features.append(img)
        labels.append(label_encoded)


In [ ]:
features

In [ ]:
labels


In [ ]:
features_array=np.array(features)
labels_array=np.array(labels)

In [ ]:
labels_one_hot=pd.get_dummies(labels_array,dtype=int)

In [ ]:
labels_one_hot

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(features_array, labels_array, test_size=0.2, random_state=42)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.25, random_state=1)

In [ ]:
len(X_train)

In [ ]:
len(X_val)

In [ ]:
len(X_test)

In [ ]:
y_test

In [ ]:
from tensorflow.keras import layers, Input, Model
from tensorflow.keras.models import Sequential
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.applications.resnet50 import preprocess_input as pp_i
from tensorflow.keras.layers import RandomFlip, RandomRotation, Dense, Dropout
from tensorflow.keras.losses import CategoricalCrossentropy
from tensorflow.keras.optimizers import Adam

In [ ]:
data_augmentation=Sequential([RandomFlip("horizontal_and_vertical"),RandomRotation(0.2)])
prediction_layers=Dense(16,activation="softmax")

In [ ]:
resnet_model=ResNet50(include_top=False,pooling='avg',weights='imagenet')
resnet_model.trainable=False
preprocess_input=pp_i

In [ ]:
inputs=Input(shape=())
x=data_augmentation(inputs)
x=preprocess_input(x)
x=resnet_model(x,training=False)
x=Dropout(0.2)(x)
outputs=prediction_layers(x)
model=Model(inputs,outputs)

In [ ]:
model.summary()

In [ ]:
print("Shape of y_train:", y_train.shape)
print("Shape of y_val:", y_val.shape)

In [ ]:
model.compile(optimizer=Adam(),loss=CategoricalCrossentropy(),metrics=['accuracy'])

In [ ]:
model_history=model.fit(x=X_train,y=y_train,validation_data=(X_val,y_val),epochs=10)

In [ ]:
from tensorflow.keras.utils import to_categorical

y_train_one_hot = to_categorical(y_train, num_classes=16)
y_val_one_hot = to_categorical(y_val, num_classes=16)


In [ ]:
acc=model_history.history['accuracy']
val_acc=model_history.history['val accuracy']
loss=model_history.history['loss']
val_loss=model_history.history['val accuracy']

In [ ]:
epochs_range=range(10)
plt.figure(figsize=(15,8))


plt.subplot(1,2,1)
plt.plot(epochs_range,acc,label='Training accuracy')
plt.plot(epochs_range,val acc,label='Validation accuracy')
plt.legend(loc='lower right')
plt.title('Training and Validation Accuracy')

plt.subplot(1,2,2)
plt.plot(epochs_range, loss, label='Training Loss')
plt.plot(epochs_range, val_loss, label='Validation Loss')
plt.legend(loc='upper right')
pil.title('Training and Validation Loss')

In [ ]:
model.evaluate(X_test,y_test)

In [ ]:
y_pred=model.predict(X_test)

In [ ]:
y_pred